# Introducción: Personalizar funciones de pérdida con hiperparámetros

En este pequeño ejercicio vamos a definir una clase que nos permita crear una función de pérdida con hiperparámetros personalizados. Esto nos va a permitir iterar sobre la función de pérdida cambiando el valor de los hiperparámetros definidos.

1. Importar librerías
2. Creación de arrays
3. Definimos funciones de pérdida personalizadas

# 1. Importar librerías

In [6]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.losses import Loss

# 2. Creación de arrays

In [2]:
Xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
Ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# 3. Funciones de pérdida funcionalizadas

En esta sección vamos a definir funciones y classes para la métrica de error Huber Loss.
Primero vamos a crear una "wrapped formula" que defina el hiperparámetro deseado de una fórmula dentro de otra.
En la segunda instancia vamos a deifinir uns clase de pérdida, que nos sirva para poder usar la función de pérdida directo en nuestro modelo como si estuviera integrada dentro de la librería.

In [3]:
def huber_loss_threshold(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return(tf.where(small_error, small_error_loss, big_error_loss))
    return huber_loss

In [5]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=huber_loss_threshold(threshold=1))
model.fit(Xs, Ys, epochs=500, verbose=0)
print(model.predict(np.array([10.0])))

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
[[18.723654]]


In [9]:
class HuberLoss(Loss):
    threshold = 1
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(small_error, small_error_loss, big_error_loss)

In [11]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=HuberLoss(threshold=1))
model.fit(Xs, Ys, epochs=500, verbose=0)
print(model.predict(np.array([10.0])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[[5.256429]]


# 4. Conclusiones

Personalizar funciones de pérdida nos ayuda a eficientar la manera en la que entrenamos los modelos.
Esto nos permite iterar los modelos entrenandolos de manera que su resultado sea más efectivo cuando buscamos predecir con una regresión.

Crear formulas wrapped o classes nos permiten iterar sobre hiperparámetros de nuestra función de pérdida.
Esto nos ayuda a iterar sobre un modelo de manera más sencilla y de manera personalizada a las metas de entrenamiento y funcionamiento del modelo.